In [2]:
import os
import sys
from pathlib import Path

project_root = Path.cwd()
sys.path.insert(0, str(project_root))

from prepare_params import prepare_production_params

print("Modules imported successfully")
print(f"Current working directory: {os.getcwd()}")


kml = "Sample Points\AfforestationSItesFixed.kml"
shp = "Sample Points\FieldPoints32_2018.shp"
tmx = "TMX\TML_pipeline_100mbuffer.kml"
cbc = "Sample Points\ColdwaterBCregion.kml"

print(kml)
print(shp)
print(tmx)
print(cbc)

Modules imported successfully
Current working directory: C:\Users\aradar\LEAF Files\LEAF_runner
Sample Points\AfforestationSItesFixed.kml
Sample Points\FieldPoints32_2018.shp
TMX\TML_pipeline_100mbuffer.kml
Sample Points\ColdwaterBCregion.kml


In [3]:
# TEMPLATE 1: Single Year Monthly Mosaics
# Use Case: Generate monthly mosaics for specific months in one year
# Example: Summer months (June, July, August) of 2023
# Customize these parameters as needed:

ProdParams = {
    # ============ REQUIRED PARAMETERS ============
    'sensor': 'S2_SR',                    # Sensor type
    'unit': 2,                   # A data unit code (1 or 2 for TOA or surface reflectance)    
    'nbYears': -1,            # positive int for annual product, or negative int for monthly product
    #'year': 2023,                          # CHANGE THIS: Year to process
    #'months': [8,9,10],                   # CHANGE THIS: Months to process (1-12)
    
    
    # ============ REGION PARAMETERS ============
    'regions': tmx,  # CHANGE THIS: Path to your KML/SHP file
    #'file_variables': ['TARGET_FID', 'AsssD_1', 'AsssD_2'], # CHANGE THIS: id, start_date, end_date (None for dates if they don't exist)
    'file_variables': ['id', 'begin', 'end'],
    #'file_variables': ['system:index', None, None],
    'regions_start_index': 3,              # CHANGE THIS: Start at this region index
    'regions_end_index': 5,             # CHANGE THIS: End at this index (None = all)
    
    # ============ BUFFER PARAMETERS ============
    #'spatial_buffer_m': 1000,             # UNCOMMENT & CHANGE: Buffer in meters around regions
    #'temporal_buffer': [[50, 90], [-10, 10], [100, 130]],          # UNCOMMENT & CHANGE: [days_before, days_after]
    #'temporal_buffer': [["2025-05-01", "2025-05-31"]], #,["2020-08-01", "2024-08-31"]],
    #'num_years': 3,
    
    
    # ============ OUTPUT PARAMETERS ============
    'resolution': 30,                      # Resolution in meters
    'projection': 'EPSG:3979',             # Coordinate projection
    'prod_names': ['LAI'],    #['LAI', 'fCOVER', 'fAPAR', 'Albedo'], 
    'bands': ['blue', 'green', 'red', 'rededge1', 'rededge2', 'rededge3', 'nir08', 'swir16', 'swir22'],
    'out_folder': 'E:/S2_mosaics_runner_2026/TMX/S2_LAI_TMX_august_2020_30m',
    'out_datatype': 'int16'
}

CompParams = {
  "number_workers":10,     
  "debug"       : True,
  "entire_tile" : False, 
  "nodes"       : 1,
  "node_memory" : "50G",
  'chunk_size': {'x': 512, 'y': 512}
}


print("\nconfigured")


configured


In [4]:
# Run the parameter preparation
result = prepare_production_params(ProdParams, CompParams)
'''
if result:
    print("\n" + "="*80)
    print(" SUCCESS - Parameters validated and ready!")
    print("="*80)
    print(f"\n Summary:")
    print(f"  Regions: {len(result['ProdParams']['regions'])}")
    print(f"  Time windows: {len(result['ProdParams']['start_dates'])}")
    print(f"  Total tasks: {len(result['ProdParams']['regions']) * len(result['ProdParams']['start_dates'])}")
    print(f"  Output folder: {result['ProdParams']['out_folder']}")
    print(f"\n Processing log saved to: {result['ProdParams']['out_folder']}/polygon_processing_log.csv")
    print("\n" + "="*80)
    print("Next step: Call Production.py with the validated parameters")
    print("="*80)
else:
    print("\n" + "="*80)
    print(" FAILED - Parameter validation failed")
    print("="*80)
    print("Check the error messages above for details.") '''


PARAMETER PREPARATION

Step 1/4: Validating input parameters...
✓ All parameters valid

Step 2/4: Loading regions from files...
<handle_regions_from_file> Detected file-based regions input: TMX\TML_pipeline_100mbuffer.kml
<handle_regions_from_file> Loading regions...
<handle_regions_from_file> Loaded 3 regions from file

Step 3/4: Skipping temporal window generation (no year/months or start_dates/end_dates found)
           Region-specific dates from file will be used if available.

Step 4/4: Validating polygons...

POLYGON VALIDATION AND FILTERING

Processing log saved to: E:/S2_mosaics_runner_2026/TMX/S2_LAI_TMX_august_2020_30m\polygon_processing_log.csv

Summary:
   Total region-date combinations: 3
   Valid for processing: 3
   Skipped (zero area): 0
{'regionID_10003': {'type': 'Polygon', 'coordinates': [[[-120.222561787557, 51.4782567013907], [-120.22253033011, 51.4784706854544], [-120.222374225606, 51.478768052375], [-120.222128472363, 51.478996965121], [-120.221959723363, 51.47

'\nif result:\n    print("\n" + "="*80)\n    print(" SUCCESS - Parameters validated and ready!")\n    print("="*80)\n    print(f"\n Summary:")\n    print(f"  Regions: {len(result[\'ProdParams\'][\'regions\'])}")\n    print(f"  Time windows: {len(result[\'ProdParams\'][\'start_dates\'])}")\n    print(f"  Total tasks: {len(result[\'ProdParams\'][\'regions\']) * len(result[\'ProdParams\'][\'start_dates\'])}")\n    print(f"  Output folder: {result[\'ProdParams\'][\'out_folder\']}")\n    print(f"\n Processing log saved to: {result[\'ProdParams\'][\'out_folder\']}/polygon_processing_log.csv")\n    print("\n" + "="*80)\n    print("Next step: Call Production.py with the validated parameters")\n    print("="*80)\nelse:\n    print("\n" + "="*80)\n    print(" FAILED - Parameter validation failed")\n    print("="*80)\n    print("Check the error messages above for details.") '

In [5]:
# View processing log (if pandas is available)
try:
    import pandas as pd
    if result and result['processing_log'] is not None:
        print("\n Processing Log Preview:")
        print("=" * 80)
        display(result['processing_log'].head(10))
        print(f"\nTotal entries: {len(result['processing_log'])}")
except ImportError:
    print("pandas not available - install it to view the processing log in this cell")
    print("You can still view the CSV file directly in the output folder")


 Processing Log Preview:


,region_id,area_m2,will_process,status,skip_reason,timestamp
0,regionID_10003,43246577.46,True,QUEUED,N/A,2026-02-18T08:09:17.775701
1,regionID_10004,10012330.28,True,QUEUED,N/A,2026-02-18T08:09:17.775701
2,regionID_10005,2883117.45,True,QUEUED,N/A,2026-02-18T08:09:17.775701



Total entries: 3


In [6]:
import os
import sys
import geopandas as gpd

#Get the absolute path to the parent of current working directory 
cwd    = os.getcwd()
source_path = os.path.join(cwd, 'source')
sys.path.append(source_path)
sys.path

from Production import main


In [7]:
INVALID_KEYS = {
    "regions_start_index",
    "regions_end_index",
    "spatial_buffer_m",
    "temporal_buffer",
    "num_years",
    "file_variables"
}

for key in list(ProdParams.keys()):
    if key in INVALID_KEYS:
        ProdParams.pop(key)


In [8]:
print(ProdParams)

{'sensor': 'S2_SR', 'unit': 2, 'nbYears': -1, 'regions': {'regionID_10003': {'type': 'Polygon', 'coordinates': [[[-120.222561787557, 51.4782567013907], [-120.22253033011, 51.4784706854544], [-120.222374225606, 51.478768052375], [-120.222128472363, 51.478996965121], [-120.221959723363, 51.479107816121], [-120.221206237297, 51.4796027774781], [-120.221204690727, 51.4796037913661], [-120.220934773541, 51.4797803849848], [-120.220828737302, 51.4798826364771], [-120.219312473635, 51.4816976155594], [-120.219045184787, 51.4820199895363], [-120.219428814768, 51.4845290070788], [-120.219396776153, 51.4849719713335], [-120.219365308807, 51.4850751090129], [-120.22053762687, 51.4864554708345], [-120.220725488195, 51.4867907657448], [-120.222739972195, 51.4929243517448], [-120.222788275306, 51.4932934032777], [-120.222641961306, 51.4958554532777], [-120.222543877132, 51.4962337304066], [-120.221273747744, 51.4988606627083], [-120.22117563546, 51.5000218500732], [-120.221061021965, 51.501378238935

In [ ]:
main(ProdParams, CompParams)


<get_time_window> one of required keys is not exist!!
<LEAF_production> All input parameters =  {'sensor': 'S2_SR', 'unit': 2, 'nbYears': -1, 'regions': {'regionID_10003': {'type': 'Polygon', 'coordinates': [[[-120.222561787557, 51.4782567013907], [-120.22253033011, 51.4784706854544], [-120.222374225606, 51.478768052375], [-120.222128472363, 51.478996965121], [-120.221959723363, 51.479107816121], [-120.221206237297, 51.4796027774781], [-120.221204690727, 51.4796037913661], [-120.220934773541, 51.4797803849848], [-120.220828737302, 51.4798826364771], [-120.219312473635, 51.4816976155594], [-120.219045184787, 51.4820199895363], [-120.219428814768, 51.4845290070788], [-120.219396776153, 51.4849719713335], [-120.219365308807, 51.4850751090129], [-120.22053762687, 51.4864554708345], [-120.220725488195, 51.4867907657448], [-120.222739972195, 51.4929243517448], [-120.222788275306, 51.4932934032777], [-120.222641961306, 51.4958554532777], [-120.222543877132, 51.4962337304066], [-120.221273747

C:\Users\aradar\AppData\Local\anaconda3\envs\leaftoolbox2\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 32.06 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
